# Преамбула

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import torch
import torchvision
from torch import nn, optim
import torch.nn.functional as F
from torchvision import transforms as T

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
from tqdm import tqdm
from PIL import Image

import os
import shutil
from random import shuffle

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

### Класс модели

In [ ]:
class MOLI_Net(nn.Module):
    def __init__(self, img_channels=3, num_classes=6):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=img_channels, out_channels=64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        )
        self.head = nn.Sequential(
            nn.Linear(in_features=512, out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(in_features=4096, out_features=num_classes)
        )

    def forward(self, x):
        out = self.encoder(x)
        out = nn.Flatten()(out)
        out = self.head(out)

        return out

In [ ]:
model = MOLI_Net()
model.load_state_dict(torch.load('/content/drive/MyDrive/color_model.h5'))
model.to(device)

MOLI_Net(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Класс даталоадера

In [ ]:
for_resize = (32, 32)
base_transforms = [T.Resize(for_resize), T.ToTensor()]

labels = {
    'binom': 0,
    'geom': 1,
    'poisson': 2,
    'norm': 3,
    'pareto': 4,
    'vigner': 5
}

mir_labels = {
    0: 'binom',
    1: 'geom',
    2: 'poisson',
    3: 'norm',
    4: 'pareto',
    5: 'vigner'
}

In [ ]:
class dataset_statistic(torch.utils.data.Dataset):
    def __init__(self, root_dir, transforms=None):
        self.root_dir = root_dir
        self.transforms = transforms
        self.files = os.listdir(self.root_dir)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.files[index])
        image = Image.open(img_path)
        label = labels[img_path.split('/')[-1].split('_')[0]]

        if self.transforms:
            for t in self.transforms:
                image = t(image)
        return (image, torch.tensor(label))

# Функция для получения ответа

1. Переместите файл расширения `.h5` на `google disk: /content/drive/MyDrive`
2. Переключитесь на среду выполнения -- `GPU`
3. Выполните код во всех ячейках выше
4. Для классификации изображений перенесите их на `google disk`, а путь к папке с изображениями запишите в переменную `test`
5. Выполните код в ячейке с вызовом функции `give_answers(path)`

In [ ]:
def give_answers(path):
    for_resize = (32, 32)
    base_transforms = [T.Resize(for_resize), T.ToTensor()]

    data = dataset_statistic(path, transforms=base_transforms)
    data_loader = torch.utils.data.DataLoader(data, shuffle=False)

    true_answers, pred_answers = [], []

    model.eval()
    for images, targets in data_loader:
        images = images.to(device)
        logits = model(images)

        target_pred = torch.max(logits, dim=1)[1].cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()

        true_answers += [mir_labels[x] for x in targets]
        pred_answers += [mir_labels[x] for x in target_pred]

        print(f'TRUE: {mir_labels[targets[0]]}, PRED: {mir_labels[target_pred[0]]}')

    return pred_answers

In [ ]:
user_images = '/content/drive/MyDrive/test'

In [ ]:
answers = give_answers(user_images)